In [303]:
import evidently
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import ColumnSummaryMetric, ColumnQuantileMetric, ColumnDriftMetric
from evidently.test_suite import TestSuite
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset, DataDriftTestPreset
from evidently.tests import TestNumberOfDriftedColumns
from evidently.ui.workspace import Workspace
from evidently.ui.workspace import WorkspaceBase, RemoteWorkspace
from evidently.ui.dashboards import DashboardPanelCounter, ReportFilter, PanelValue, PlotType, CounterAgg
from abc import ABC, abstractmethod
import pandas as pd


In [341]:

class GenerateReport(ABC):
    @abstractmethod
    def create_report(self, workspace, project, reference, current):
        pass

class DataDriftReport(GenerateReport):
    def create_report(self, workspace, project, reference, current):
        data_drift_report = Report(metrics=[DataDriftPreset()], timestamp=datetime.datetime.now())
        data_drift_report.run(reference_data=reference, current_data=current)
        workspace.add_report(project_id=project.id, report=data_drift_report)

class TargetDriftReport(GenerateReport):
    def create_report(self, workspace, project, reference, current):
        pass

class DataQualityReport(GenerateReport):
    def create_report(self, workspace, project, reference, current):
        data_quality_report = Report(metrics=[DataQualityPreset()])
        data_quality_report.run(reference_data=reference, current_data=current)
        workspace.add_report(project_id=project.id, report=data_quality_report)

class RegressionReport(GenerateReport):
    def create_report(self, workspace, project, reference, current):
        regression_performance_report = Report(metrics=[RegressionPreset()])
        regression_performance_report.run(reference_data=reference, current_data=current)
        workspace.add_report(project_id=project.id, report=regression_performance_report)

class ClassificationReport(GenerateReport):
    def create_report(self, workspace, project, reference, current):
        pass

class DataDriftTestReport(GenerateReport):
    def create_report(self, workspace, project, reference, current):
        data_drift_test_suite = TestSuite(tests=[DataDriftTestPreset(), TestNumberOfDriftedColumns()])
        data_drift_test_suite.run(reference_data=reference, current_data=current)
        workspace.add_test_suite(project_id=project.id, test_suite= data_drift_test_suite)
 


class Monitoring:
    def __init__(self, strategy: DataDriftReport):
        self._strategy = strategy
        self._workspace = None
        self._project = None

    def create_workspace(self, name:str):
        self._workspace = Workspace.create(name)
        return self._workspace
    
    def create_project(self, project_name:str, workspace: WorkspaceBase = None):
        if(self._workspace is None):
            self._workspace = workspace
        self._project = self._workspace.create_project(project_name)
        return self._project

    @property
    def current_strategy(self):
        return self._strategy
        
    @current_strategy.setter
    def set_strategy(self, strategy: DataDriftReport):
        self._strategy = strategy

    def execute_strategy(self, reference: pd.DataFrame, current: pd.DataFrame, workspace: WorkspaceBase = None, column_mapping: ColumnMapping = None):
        if(self._workspace is None):
            self._workspace = workspace
        self._strategy.create_report(self._workspace, self._project, reference, current)

    def add_dashboard_panel(self, project: evidently.ui.base.Project, panel_type: str, **kwargs):
        match panel_type:
            case "Counter":
                project.dashboard.add_panel(
                    DashboardPanelCounter(
                        title=kwargs["title"],
                        filter=ReportFilter(metadata_values={}, tag_values=[]),
                        value=PanelValue(
                            metric_id=kwargs["metric_id"],
                            field_path=kwargs["field_path"],
                            legend=kwargs["legend"],
                        ),
                        text=kwargs["text"],
                        agg=CounterAgg.LAST,
                        size=1,
                    )
                )

            case "Plot":
                project.dashboard.add_panel(
                    DashboardPanelPlot(
                        title=kwargs["title"],
                        filter=ReportFilter(metadata_values={}, tag_values=[]),
                        values=[
                            PanelValue(
                                metric_id=kwargs["metric_id"],
                                metric_args=kwargs["metric_args"],
                                field_path=kwargs["field_path"],
                                legend=kwargs["legend"]
                            ),
                        ],
                        plot_type=kwargs["plot_type"],
                        size=WidgetSize.HALF
                    )
                )

            case "TestSuite":
                project.dashboard.add_panel(
                    DashboardPanelTestSuite(
                        title="All tests: detailed",
                        filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=True),
                        size=WidgetSize.HALF,
                        panel_type=TestSuitePanelType.DETAILED,
                        time_agg="1D",
                    )
                )

            case _:
                print("Specified panel type not defined!")
                
        project.save()

    def delete_dashboard(self, project: evidently.ui.base.Project):
        project.dashboard.panels = []
        project.save()





#ws.add_report
#ws.add_test_suite


In [342]:
monitoring = Monitoring(DataDriftReport())
monitoring.current_strategy

In [327]:
ws = monitoring.create_workspace("getting started evidently")
project = monitoring.create_project("proj1", ws)
project

Project(id=UUID('01940284-76c4-75a8-9e3f-57a73a9b0161'), name='proj1', description=None, dashboard=DashboardConfig(name='proj1', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, org_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 12, 26, 16, 0, 23, 172871))

In [339]:
ws.create("getting started evidently")
project_list = ws.search_project(project_name="proj1")
if(len(project_list) == 0):
    project = self._workspace.create_project(project_name)
else:
    project = project_list[0]

project

Project(id=UUID('01940279-3f83-7c46-aba1-c9a9c7e87d86'), name='proj1', description=None, dashboard=DashboardConfig(name='proj1', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, org_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 12, 26, 15, 48, 8, 131573))

In [42]:
monitoring.set_strategy = DataQualityReport()

In [56]:
ws.list_projects()

[Project(id=UUID('0193f833-9ad2-719d-8aac-d1c86d56c9af'), name='proj1', description=None, dashboard=DashboardConfig(name='proj1', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, org_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 12, 24, 15, 55, 51, 826940)),
 Project(id=UUID('0193f226-b637-74fd-9135-5edb2bfc5303'), name='live monitoring', description=None, dashboard=DashboardConfig(name='live monitoring', panels=[DashboardPanelPlot(type='evidently:dashboard_panel:DashboardPanelPlot', id=UUID('0193f23d-163e-73c3-9dd0-14782cc11ff8'), title='sample_panel', filter=ReportFilter(metadata_values={}, tag_values=['quality'], include_test_suites=False), size=<WidgetSize.FULL: 2>, values=[PanelValue(field_path='current.share_in_range', metric_id='ColumnValueRangeMetric', metric_fingerprint=None, metric_args={}, legend='current'), PanelValue(field_path='reference.share_in_range', metric_id='ColumnValueRangeMetric', metric_fingerprint=None, metric_args={}, lege

In [297]:
column_mapping = ColumnMapping()

In [301]:
monitoring.execute_strategy( [], [], ws, column_mapping)

NameError: name 'datetime' is not defined

In [ ]:
monitoring.delete_dashboard